In [1]:
#NDWI load packages

import datacube

import sys
sys.path.insert(1, '../Tools/')
from dea_tools.dask import create_local_dask_cluster
from dea_tools.plotting import rgb


In [2]:
# connect to datacube

dc = datacube.Datacube(app="07_Parallel_processing_with_Dask")

In [3]:
#Enable Dask

client = create_local_dask_cluster(return_client=True)

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/akjournalism@gmail.com/proxy/8787/status,
Dashboard: /user/akjournalism@gmail.com/proxy/8787/status,Workers: 1
Total threads: 2,Total memory: 12.21 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:36353,Workers: 1
Dashboard: /user/akjournalism@gmail.com/proxy/8787/status,Total threads: 2
Started: Just now,Total memory: 12.21 GiB
Comm: tcp://127.0.0.1:36039,Total threads: 2
Dashboard: /user/akjournalism@gmail.com/proxy/38219/status,Memory: 12.21 GiB
Nanny: tcp://127.0.0.1:44047,


In [ ]:
# List Sentinel-2 Surface Reflectance products available in DEA

dc_products = dc.list_products()
dc_products.loc[['ga_s2am_ard_3', 'ga_s2bm_ard_3']]

In [ ]:
dc_measurements = dc.list_measurements()
dc_measurements.loc[['ga_s2am_ard_3']]

In [ ]:
dc_measurements = dc.list_measurements()
dc_measurements.loc[['ga_s2bm_ard_3']]

In [4]:
# Lazy load data 

lazy_ds = dc.load(product=["ga_ls5t_ard_3", "ga_ls8c_ard_3",
                  "ga_ls9c_ard_3"],
                  measurements=["nbart_green", "nbart_swir_2", "oa_fmask"],
                  x=(149.36, 150.2),
                  y=(-37.00, -37.5),
                  time=('2020-01-14', '2020-04-14'),
                  dask_chunks={"time": 1, "x": 2048, "y": 2048})
resampling={
                   "fmask": "nearest",
                   "*": "bilinear"
               },



lazy_ds

<xarray.Dataset>
Dimensions:       (time: 18, y: 2171, x: 2727)
Coordinates:
  * time          (time) datetime64[ns] 2020-01-16T23:50:51.864078 ... 2020-0...
  * y             (y) float64 -4.149e+06 -4.149e+06 ... -4.214e+06 -4.214e+06
  * x             (x) float64 1.538e+06 1.538e+06 ... 1.62e+06 1.62e+06
    spatial_ref   int32 3577
Data variables:
    nbart_green   (time, y, x) int16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    nbart_swir_2  (time, y, x) int16 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    oa_fmask      (time, y, x) uint8 dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
Attributes:
    crs:           EPSG:3577
    grid_mapping:  spatial_ref

In [ ]:
resample

In [ ]:
# view dask chunks

lazy_ds.nbart_red.data


In [5]:
# Import tools to mask data for cloud

import scipy.ndimage
import xarray
import numpy
import datacube
from datacube.utils.masking import make_mask
from datacube.utils.masking import mask_invalid_data
from odc.algo import mask_cleanup

import sys
sys.path.insert(1, '../Tools/')
from dea_tools.plotting import rgb

In [6]:
# app

dc = datacube.Datacube(app="Masking_data")

# flags

lazy_ds.oa_fmask.attrs["flags_definition"]

{'fmask': {'bits': [0, 1, 2, 3, 4, 5, 6, 7],
  'values': {'0': 'nodata',
   '1': 'valid',
   '2': 'cloud',
   '3': 'shadow',
   '4': 'snow',
   '5': 'water'},
  'description': 'Fmask'}}

In [ ]:
# plot

lazy_ds.oa_fmask.plot(col="time", col_wrap=4)

In [7]:
# Create the mask based on "valid" pixels

clear_mask = make_mask(lazy_ds.oa_fmask, fmask="valid")

# Apply the mask

clear = lazy_ds.where(clear_mask)


In [ ]:
# Plot
clear_mask.plot(col="time", col_wrap=4)

In [8]:
# Identify pixels that are either "valid", "water" or "snow"

cloud_free_mask = (
    make_mask(lazy_ds.oa_fmask, fmask="valid") | 
    make_mask(lazy_ds.oa_fmask, fmask="water") | 
    make_mask(lazy_ds.oa_fmask, fmask="snow")
)

# Apply the mask
cloud_free = lazy_ds.where(cloud_free_mask)

In [9]:
# Set invalid nodata pixels to NaN

valid_data = lazy_ds.oa_fmask.plot
valid_data

In [10]:
# Identify pixels that are either "cloud" or "cloud_shadow"

cloud_shadow_mask = (
    make_mask(lazy_ds.oa_fmask, fmask="cloud") | 
    make_mask(lazy_ds.oa_fmask, fmask="shadow")
)


In [ ]:
# Plot cloud shadow mask

cloud_shadow_mask.plot(col="time", col_wrap=4)

In [11]:
# Dilate all cloud and cloud shadow pixels by 5 pixels in all directions

cloud_shadow_buffered = mask_cleanup(mask=cloud_shadow_mask,
                                     mask_filters=[("dilation", 5)])

In [ ]:
# Plot cloud shadow buffered

cloud_shadow_buffered.plot(col="time", col_wrap=4)

In [12]:
# Apply the mask

buffered_cloud_free = lazy_ds.where(~cloud_shadow_buffered)

In [13]:
# Set invalid nodata pixels to NaN

valid_data = mask_invalid_data(buffered_cloud_free)

In [ ]:
valid_data.nbart_red.isel(time=1)


In [ ]:
valid_data

In [14]:
# calc mNDWI

band_diff = valid_data.nbart_green - valid_data.nbart_swir_2
band_sum = valid_data.nbart_green + valid_data.nbart_swir_2

valid_data["mndwi"] = band_diff / band_sum

In [ ]:
valid_data

In [ ]:
#prob not necessary
valid_data.mndwi

In [15]:
del buffered_cloud_free, cloud_shadow_buffered, cloud_free, clear_mask

In [16]:
# make data an XArray from Dask

ndwi_load = valid_data.mndwi.load()
ndwi_load

<xarray.DataArray 'mndwi' (time: 18, y: 2171, x: 2727)>
array([[[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        ...,
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan]],

       [[        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
        [        nan,         nan,         nan, ...,         nan,
                 nan,         nan],
...
        [-0.37914692, -0.30779392, -0.33096927, ...,         nan,
                 nan,         nan],
        [-0.38354807, -0.35267857, -0.32387707, ...,         nan,
                 nan,         nan],
        [-0.37488708, -0.4106981 , -0.35294118, ...,         nan,
                 nan,         nan]],

       [[        nan,         nan,         nan, ...,  0.78571429,
          0.81065089,  0.7804878 ],
        [        nan,         nan,         nan, ...,  0.78313253,
          0.82608696,  0.80838323],
        [        nan,         nan,         nan, ...,  0.77245509,
          0.80368098,  0.80116959],
        ...,
        [        nan,         nan,         nan, ...,  0.83625731,
          0.86046512,  0.80346821],
        [        nan,         nan,         nan, ...,  0.80952381,
          0.78947368,  0.82352941],
        [        nan,         nan,         nan, ...,  0.84795322,
          0.83625731,  0.8452381 ]]])
Coordinates:
  * time         (time) datetime64[ns] 2020-01-16T23:50:51.864078 ... 2020-04...
  * y            (y) float64 -4.149e+06 -4.149e+06 ... -4.214e+06 -4.214e+06
  * x            (x) float64 1.538e+06 1.538e+06 1.538e+06 ... 1.62e+06 1.62e+06
    spatial_ref  int32 3577

In [ ]:
# plot data 

loaded_ds.mndwi.plot(col='time', col_wrap=5)

In [17]:
# create binary water not water data

import rasterio
import xarray as xr

# Extract the NDWI data variable COULD BE WRONG WORKS WITHOUT
ndwi_variable = valid_data.mndwi

# Set the threshold
threshold = 0.3  # Adjust as per your requirement

# Create a binary water or no water raster
binary_data = ndwi_variable > threshold

#binary_data = binary_date.dtype('uint8')

In [18]:
# Calculate total wetness through time
wet_pixels = binary_data.sum(dim=['time'])

In [ ]:
wet_pixels

In [ ]:
#not necessary
wet_pixels.plot(size=5, cmap='Blues')

In [19]:
wet_pixels.attrs = valid_data.attrs

In [20]:
#wet_pixels = wo.isel(time=-0).to_array()

from datacube.utils.cog import write_cog

write_cog (geo_im=wet_pixels,
          fname='./Landsat_bin_NDWI_sum_slice_5.tif',
          overwrite=True)

PosixPath('Landsat_bin_NDWI_sum_slice_5.tif')

In [ ]:
#CLOSE DASK

client.close()

In [ ]:
del valid_data, wet_pixels